In [1]:
pip install requests

In [5]:
# LangChain core
!pip install langchain

In [6]:
# PDF loading (PyMuPDF)
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 76.9 MB/s eta 0:00:00


In [7]:
# FAISS for vector storage
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 59.6 MB/s eta 0:00:00


In [8]:
# HuggingFace embeddings
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 837.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [9]:
# Perplexity integration (via langchain_community)
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00


In [3]:
import requests

# Set up the API endpoint and headers
api_key = "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"

url = "https://api.perplexity.ai/chat/completions"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# Define the request payload
payload = {
    "model": "sonar-pro",
    "messages": [
        {"role": "user", "content": "Which team won the FIFA Club World Cup 2025"}
    ]
}

# Make the API call
response = requests.post(url, headers=headers, json=payload)

# Print the AI's response
print(response.json()["choices"][0]['message']['content'])

The **2025 FIFA Club World Cup** was won by **Chelsea**, who defeated Paris Saint-Germain 3–0 in the final held at MetLife Stadium in East Rutherford, New Jersey[1][2][3][4][5]. This marked Chelsea's second Club World Cup title and their first since 2021[4].

Key details:
- Chelsea's victory was secured by a commanding first-half performance, with **Cole Palmer** scoring twice and assisting João Pedro for the third goal[1].
- Paris Saint-Germain were unable to recover, especially after a late red card to João Neves in the 85th minute[1].
- The final served as a major event in the lead-up to the 2026 FIFA World Cup, as it was held at the same venue[1].

Chelsea capped off a strong season with their international double, having also won the UEFA Conference League earlier in the year[1][4].


In [10]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.chat_models import ChatPerplexity
from langchain.chains import RetrievalQA
import os

In [12]:
#Load
loader = PyMuPDFLoader("/content/sample_data/Lan (Alan) Dao - Resume.pdf")
documents = loader.load()

#Split
splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=500)
chunks = splitter.split_documents(documents)

#Embbedding
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False} #False Euclidean, True cosine similarity
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

#Vector Store
vector_store = FAISS.from_documents(chunks, hf)
vector_store.save_local("faiss_index_open")

#Retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

#LLM
llm = ChatPerplexity(
    model="sonar-pro",
    pplx_api_key = "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",
    temperature=0.2
)

# QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True
)

/tmp/ipython-input-12-2943583078.py:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-12-2943583078.py:27: LangChainDeprecationWarning: The class `ChatPerplexity` was deprecated in LangChain 0.3.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-perplexity package and should be used instead. To use it run `pip install -U :class:`~langchain-perplexity` and import as `from :class:`~langchain_perplexity import ChatPerplexity``.
  llm = ChatPerplexity(


In [21]:
#query = "When is Lan Dao expected to graduate?"
query = "What are Lan Dao’s certifications from the resume?"
#query = "Dont make up the answer if you dont know. Question: What are Lan Dao's top skills?"
response = qa_chain.invoke(query)

In [22]:
print("Response", response['result'])
print("############################")
for i, doc in enumerate(response['source_documents']):
    print(f"######DOC {i+1}######")
    print(doc)

Response Lan Dao’s certifications from the resume are:

- **IBM Data Science Professional Certificate**
- **IBM Data Analyst Certificate**

These are listed under the "SKILLS AND CERTIFICATIONS" section of the resume provided. No other certifications are mentioned in the resume content.
############################
######DOC 1######
page_content='LAN (ALAN) DAO  
ldd0711@mavs.uta.edu | 682-306-5752 | https://www.linkedin.com/in/landao1203/ 
 
EDUCATION 
University of Texas - Arlington​
​
​
​
​
​
​
​
  Aug 2023 - May 2027  
GPA: 3.95 
Major: Applied Statistic Data Science 
Awards: Dean’s List Fall 2023, Year 2024 | Maverick Academic Scholarship  
 
SKILLS AND CERTIFICATIONS 
Programming Languages: Python (Pandas, NumPy, Scikit-learn, PyTorch, OpenCV), R, SAS studio 
Database Management: SQL (MySQL, SQLite, PostgreSQL) 
Data Visualization: Matplotlib, Seaborn, Plotly, Power BI, Tableau, Microsoft Excel 
Certificates: IBM Data Science Professional | IBM Data Analyst  
                    